In [6]:
from air_hockey_challenge.framework.air_hockey_challenge_wrapper import AirHockeyChallengeWrapper
from air_hockey_challenge.framework.challenge_core import ChallengeCore
from air_hockey_challenge.framework.agent_base import AgentBase
from examples.control.hitting_agent import build_agent as build_agent_hitting

from mushroom_rl.utils.dataset import parse_dataset, select_random_samples
from mushroom_rl.policy import GaussianTorchPolicy

import torch
import torch.nn as nn

from tqdm import tqdm

import pickle

import numpy as np

use_cuda = torch.cuda.is_available()
device = 'cuda' if use_cuda else 'cpu'
print(f"Cuda: {use_cuda}")

Cuda: True


In [7]:
class Network(nn.Module):
    def __init__(self, input_shape, output_shape, n_features, **kwargs):
        super(Network, self).__init__()

        self._h1 = nn.Linear(input_shape[0], n_features)
        self._h2 = nn.Linear(n_features, n_features)
        self._h3 = nn.Linear(n_features, output_shape[0])

        nn.init.xavier_uniform_(self._h1.weight,
                                gain=nn.init.calculate_gain('tanh'))
        nn.init.xavier_uniform_(self._h2.weight,
                                gain=nn.init.calculate_gain('tanh'))
        nn.init.xavier_uniform_(self._h3.weight,
                                gain=nn.init.calculate_gain('linear'))

    def forward(self, obs, **kwargs):
        features1 = torch.tanh(self._h1(torch.squeeze(obs, 1).float()))
        features2 = torch.tanh(self._h2(features1))
        return self._h3(features2)

# policy can only output 1D actions (6,)
policy = GaussianTorchPolicy(Network, (12,), (6,), std_0=1., n_features=64, use_cuda=use_cuda)

In [ ]:
from air_hockey_challenge.framework.evaluate_agent import generate_init_states


def generate_init_joint_states(mdp ,env: str = "3dof-hit"):

    # create initial state
    obs = generate_init_states(env=env, n_episodes=1, n_parallel_cores=1)
    obs = obs[0].flatten()

    # get joint position limits
    min_pos_limits = mdp.env_info['robot']['joint_pos_limit'][0, :]
    max_pos_limits = mdp.env_info['robot']['joint_pos_limit'][1, :]

    # generate random joint positions
    random_joint_pos = np.random.uniform(low=min_pos_limits, high=max_pos_limits, size=(3,))

    # save random joint positions in the initial state
    obs[mdp.env_info['joint_pos_ids']] = random_joint_pos

    obs = obs.reshape((1,-1))

    return obs

In [ ]:
env = "3dof-hit"

mdp = AirHockeyChallengeWrapper(env)
mdp.reset()

agent = build_agent_hitting(mdp.env_info)
core = ChallengeCore(agent, mdp)

n_expert_traj = 1000
dataset_path = f'dataset/hit_{n_expert_traj}.pkl'

In [ ]:
dataset = []
success_traj = 0
for i in tqdm(range(n_expert_traj)):
    # initial_state = generate_init_joint_states(mdp, env=env)
    # new_traj = core.evaluate(initial_states=initial_state)
    new_traj = core.evaluate(n_episodes=1)
    _, _, _, _, absorbed, _ = parse_dataset(new_traj)
    if absorbed[-1]: # we do not want examples where the "expert" did not score
        dataset += new_traj
        success_traj += 1
print(f"Dataset size: {len(dataset)} from {success_traj} trajectories")

with open(dataset_path, 'wb') as f:
    pickle.dump(dataset, f)

In [ ]:
from air_hockey_challenge.utils.replay_dataset import replay_dataset
replay_dataset(env, dataset_path)

In [ ]:
def train_bc(policy, dataset, n_epochs, batch_size, n_batches_per_epoch, lr=0.0005):
    optimizer = torch.optim.Adam(policy.parameters(), lr=lr)
    
    pbar = tqdm(range(n_epochs), desc='Epoch')
    logs = dict(loss=[], epoch=[])
    for iter in pbar:
        avg = []
        for i in range(n_batches_per_epoch):
            states, actions, rewards, next_states, absorbing, last  = select_random_samples(dataset, batch_size, parse=True)
            states_t = torch.from_numpy(states).to(device)
            expert_actions_t = torch.from_numpy(actions.reshape((batch_size,-1))).to(device)
            
            optimizer.zero_grad()
            loss = -policy.log_prob_t(states_t, expert_actions_t).mean()
            loss.backward()
            optimizer.step()
            avg.append(loss.item())
        
        avg_loss = np.mean(avg)
        pbar.set_postfix({'loss': avg_loss})
        logs['loss'].append(avg_loss)
        logs['epoch'].append(iter)
    return logs

In [ ]:

dataset = np.load(f"dataset/hit_{n_expert_traj}.pkl", allow_pickle=True)

logs = train_bc(policy, dataset, 100, 100, 100)
policy.save('dataset/hit_500_policy')

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.plot(logs['epoch'], logs['loss'])

In [9]:
from mushroom_rl.algorithms.actor_critic import PPO

class PPOAgent(PPO): # Just to fix the action shape
    def draw_action(self, state):
        return super().draw_action(state).reshape(2,3)
    
    def fit(self, dataset, **info):
        for i in range(len(dataset)):
            dataset[i] = list(dataset[i])
            dataset[i][1] = dataset[i][1].flatten()
        return super().fit(dataset, **info)

In [11]:
class BCAgent(AgentBase):
    def __init__(self, env_info, policy, **kwargs):
        super().__init__(env_info, **kwargs)
        self.policy = policy

    def reset(self):
        pass

    def draw_action(self, observation):
        return self.policy.draw_action(observation).reshape(2,3)
    
critic_params = dict(network=Network,
                        optimizer={'class': torch.optim.Adam,
                                'params': {}},
                        loss=torch.nn.functional.mse_loss,
                        n_features=64,
                        batch_size=64,
                        input_shape=(12,),
                        output_shape=(1,), 
                        use_cuda = False)

alg_params = dict(actor_optimizer={'class': torch.optim.Adam,
                                    'params': {}},
                    critic_params=critic_params,
                    n_epochs_policy = 1,
                    batch_size = 5,
                    eps_ppo = 0.01,
                    lam = 0.95,
                 )

policy_params = dict(
    std_0=1.,
    n_features=64,
    use_cuda=False
)

policy = GaussianTorchPolicy(Network,
                                (12,),
                                (6,),
                                **policy_params)
# policy.load("dataset/policy")

agent = PPOAgent(mdp.info, policy, **alg_params)
agent = agent.load('dataset/PPO/epoch_14.pkl')

env = "3dof-hit"

mdp = AirHockeyChallengeWrapper(env)
mdp.reset()

# agent = BCAgent(mdp.env_info, policy)
core = ChallengeCore(agent, mdp)

eval = core.evaluate(n_episodes=5, render=True)

: 

In [4]:
def build_agent(env_info, **kwargs):
    """
    Function where an Agent that controls the environments should be returned.
    The Agent should inherit from the mushroom_rl Agent base env.

    :param env_info: The environment information
    :return: Either Agent ot Policy
    """
    if "hit" in env_info["env_name"]:
        return BCAgent(mdp.env_info, policy)

In [5]:
from air_hockey_challenge.framework.evaluate_agent import evaluate

eval = evaluate(build_agent, 'logs/bc_agent', env_list=[env], n_episodes=50, render=False, quiet=False, n_cores=1)

Environment:        3dof-hit
Number of Episodes: 50
Success:            0.1000
Penalty:            202.5
Number of Violations: 
  joint_vel_constr  50
  ee_constr         32
  Jerk              50
  joint_pos_constr  3
  Computation Time  1
  Total             136
-------------------------------------------------

